In [ ]:
# Import necessary libraries

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# Download 20 years of financial data for these tickers.
tickers = ['AAPL','MSFT','GOOGL','AMZN','TSLA','NVDA','JPM','V','MA','PG',
           'JNJ','UNH','HD','VZ','DIS','PYPL','ADBE','NFLX','INTC','T',
           'PFE','MRK','ABBV','KO','PEP','XOM','CVX','WMT','BAC','C',
           'CRM','ABT','ACN','TXN','AVGO','COST','DHR','LLY','MCD','NKE',
           'ORCL','QCOM','SBUX','TMO','UNP','UPS','LIN','LOW','HON']

data = yf.download(tickers, start="2004-01-01", end="2025-12-31", 
                   auto_adjust=True, threads=True)['Close']

# Save and load quickly
data.to_pickle("tickers_20y.pkl")
data = pd.read_pickle("tickers_20y.pkl")

data.head()

In [ ]:
# Daily  returns
daily_returns = data.pct_change()

# Buy and hold for 1$ for each ticker. Show 5 last days.
equity_curves_df = (1 + daily_returns).cumprod() 
equity_curves_df.tail()

In [ ]:
# Plot the equity curves
plt.style.use("seaborn-v0_8-whitegrid")
fig, ax = plt.subplots(figsize=(16, 9))

final_values = equity_curves_df.iloc[-1]
final_values = final_values[equity_curves_df.count() > 4000]

# 3 best performers
top3 = final_values.nlargest(3).index   # casi siempre NVDA, AVGO, AAPL/LLY
equity_curves_df[top3].plot(ax=ax, linewidth=1.5, alpha=0.45)

# 2 worst performers
bottom2 = final_values.nsmallest(2).index  # casi siempre INTC y T o VZ
equity_curves_df[bottom2].plot(ax=ax, linewidth=1.5, alpha=0.45)

# Rest of the tickers
others = equity_curves_df.drop(columns=top3.union(bottom2))
lower = others.min(axis=1, skipna=True)
upper = others.max(axis=1, skipna=True)
ax.fill_between(others.index, lower, upper, color="gray", alpha=0.25, label="Resto (~90%)")

# SPY returns for benchmark
spy = yf.download('SPY', start="2004-01-01", end="2025-12-31", auto_adjust=True, threads=True)['Close']
spy.to_pickle("spy.pkl")
spy = pd.read_pickle("spy.pkl")

spy_returns = spy.pct_change()
spy_equity_df = (1 + spy_returns).cumprod()
spy_equity_df = spy_equity_df.reindex(equity_curves_df.index).fillna(1.0)

spy_equity_df.plot(ax=ax, linewidth=3,linestyle=":", alpha=1)

# Aesthetics
ax.set_yscale('log')
ax.set_title("Investment performance: 3 best performers, 2 worst performers and the other 90%.", fontsize=16)
ax.set_ylabel("Value of 1$ (log scale)")
ax.legend(loc="upper left")
ax.set_xlabel("Date")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate and show a table with different metrics: CAGR, Sharpe and max drawdown per stock.

# Compound annual growth rate (CAGR)
days_held = (equity_curves_df.index[-1] - equity_curves_df.index[0]).days
years_held = equity_curves_df.notna().cumsum().iloc[-1] * (days_held / 365.25) / len(equity_curves_df)

cagr = (equity_curves_df.iloc[-1] ** (1 / years_held)) - 1 

cagr

In [ ]:
# Sharpe ratio
irx = yf.download("^IRX", start="2004-01-01", end="2025-12-31", auto_adjust=True)['Close']

rf_daily = irx.reindex(daily_returns.index).ffill().bfill() / 100 / 252
rf_yearly = ((1 + rf_daily.mean()) ** 252 - 1).iloc[0]

yearly_returns = daily_returns.mean() * 252

sigma = daily_returns.std() * np.sqrt(252)

sharpe = (yearly_returns - rf_yearly) / sigma

sharpe

In [ ]:
# Maximum drawdown

highest_highs = equity_curves_df.cummax()
max_drawdowns = (equity_curves_df / highest_highs - 1).min() 

max_drawdowns

In [ ]:
# Rolling sharpe for 252 days 

rf_daily = rf_yearly / 252
excess_returns = daily_returns - rf_daily
rol_sharpe = ((excess_returns.rolling(window=252).mean() * 252) / (excess_returns.rolling(window=252).std() * np.sqrt(252)))

rol_sharpe

In [ ]:
#Table to show metrics

rolling_sharpe_latest = rol_sharpe.iloc[-1]

metrics_table = pd.DataFrame({
    'CAGR (%)': (cagr * 100).round(2),
    'Calmar ratio': (cagr / -max_drawdowns).round(2),
    'Max drawdown (%)': (max_drawdowns * 100).round(2),
    'Sharpe ratio': sharpe.round(2),
    'Rolling Sharpe 252d': rolling_sharpe_latest.round(2),
    'Current Vol 252d (%)': (daily_returns.rolling(252).std().iloc[-1] * np.sqrt(252) * 100).round(2)
})


metrics_table = metrics_table.sort_values('Rolling Sharpe 252d', ascending=False)

styled_table = (metrics_table
    .style
    .set_properties(**{'font-size': '13pt', 'font-family': 'Calibri, Arial', 'text-align': 'center'})
    .set_table_styles([
        {'selector': 'th', 
         'props': [('background-color', '#e8e8e8'), ('color', '#333333'), ('font-weight', 'bold'), ('padding', '12px')]},
        {'selector': 'td', 
         'props': [('padding', '10px'), ('border-bottom', '1px solid #ddd')]},
        {'selector': 'caption', 
         'props': [('font-size', '20pt'), ('font-weight', 'bold'), ('color', '#2b2b2b'), ('margin-bottom', '15px')]}
    ])
    .background_gradient(cmap='Blues', subset=['Calmar ratio'], low=0.4, high=1.0)
    .bar(subset=['Max drawdown (%)'], color='#ffb3b3', height=70)
    .bar(subset=['Current Vol 252d (%)'], color='#e6e6e6', height=60)
    .background_gradient(cmap='Greys', subset=['Rolling Sharpe 252d'], low=0.4, high=0.9)
    .format('{:.1f}', subset=['CAGR (%)', 'Max drawdown (%)', 'Current Vol 252d (%)'])
    .format('{:.2f}', subset=['Sharpe ratio', 'Rolling Sharpe 252d', 'Calmar ratio'])
    .set_caption("Metrics highlight")
)

display(styled_table)

In [ ]:
# Correlation heatmap

corr = daily_returns.corr()

plt.figure(figsize=(24, 20))

im = plt.imshow(corr, cmap='RdYlGn_r', vmin=-1, vmax=1, interpolation='nearest') 

cbar = plt.colorbar(im, shrink=0.7, aspect=20) # Legend
cbar.set_label('Correlation', rotation=270, labelpad=25, fontsize=16)
cbar.ax.tick_params(labelsize=14)

plt.xticks(range(len(corr.columns)), corr.columns, rotation=90, fontsize=11) # Tickers
plt.yticks(range(len(corr.index)), corr.index, fontsize=11)

plt.grid(True, which='major', color='white', linewidth=1.5)
plt.grid(False)

for i in range(len(corr)): #Colored numbers inside cells 
    for j in range(len(corr)):
        value = corr.iloc[i, j]
        color = "white" if abs(value) > 0.7 else "black"
        plt.text(j, i, f'{value:.2f}', 
                 ha='center', va='center', 
                 color=color, fontsize=9, fontweight='bold')
        
plt.title("Correlation heatmap (2004-2025)", fontsize=26, pad=40, fontweight='bold')
plt.tight_layout()
plt.show()